In [6]:
import numpy as np
import pandas as pd
import tqdm as tqdm
import torch
from nn_tools import Qfirst, Qfollowing, optimiers_list, one_epoch_layers
import torch.nn as nn
from learning_tools import peak_one, algo_p, algo_e, update

ImportError: cannot import name 'algo_p' from 'learning_tools' (c:\Users\Chardin Pierre\OneDrive - CentraleSupelec\Bureau\CS\3a\DL\RL-for-bridge-bidding\learning_tools.py)

In [ ]:
# Entraînement du modèle spécifique à chaque couche d'enchère
def learning_algo(data,max_iter=100, nb_layers=5, epsilon=0.1,
                  loss= nn.MSELoss(),batch_size=50, device='cpu'):

    D = [[torch.empty(0), torch.empty(0)] for _ in range(nb_layers)]  # Un dataset D par couche
    all_actions = list(range(36))  # 36 actions possibles
    Q_models = [Qfirst() if i == 0 else Qfollowing() for i in range(nb_layers)]
    losses=[]
    for _ in tqdm(range(max_iter)):
        bidding_history = np.zeros(36) # array de 5x36 pour enregistrer l'historique des enchères
        x1, x2, scores = peak_one(data)

        for j in range(nb_layers):
            C = np.zeros(36)  
            hand = x1 if (j+1) % 2 == 1 else x2  # main du joueur actuel

            # Calcul des coûts des actions
            for i, action in enumerate(all_actions):
                C[i] = algo_p(action,x1,x2,scores,bidding_history,Q_models,nb_layers)
            
            # state = main si première couche sinon state = main + enchère_précédente
            state = torch.tensor(hand, dtype=torch.float32) if j == 0 else torch.tensor(np.concatenate([hand, bidding_history]), dtype=torch.float32)
            D=update(D,state,C,j) #ajout de l'expérience dans le dataset D_j
            
            next_a = algo_e(Q_models[j], state, epsilon, all_actions)  # Sélection avec Q_j
            if next_a == len(bidding_history) - 1:  # si l'action est PASS
                break  
            bidding_history[next_a] = 1  # enregistrement de l'action choisie dans l'historique des enchères


        # Entraînement des modèles Q
        opti_list=optimiers_list(Q_models)
        losses.append(one_epoch_layers(Q_models, D, opti_list,loss, batch_size, device))
    return Q_models

In [5]:
data = pd.read_csv("data/slice0.csv", index_col = 0)

In [ ]:
learning_algo